https://docs.mistral.ai/

#### From Mistral docs

```
curl --location "https://api.mistral.ai/v1/chat/completions" \
     --header 'Content-Type: application/json' \
     --header 'Accept: application/json' \
     --header "Authorization: Bearer $MISTRAL_API_KEY" \
     --data '{
    "model": "mistral-tiny",
    "messages": [{"role": "user", "content": "Who is the most renowned French painter?"}]
  }'
```

# Chat W/ History & Saved Files

Run all cells.

In [51]:
import requests
import json
import os
import re

from google.colab import userdata


mistral_api_key = userdata.get('i1')

# Define the endpoint URL
endpoint_url = "https://api.mistral.ai/v1/chat/completions"

# Set the headers
headers = {
  "Content-Type": "application/json",
  "Accept": "application/json",
  "Authorization": f"Bearer {mistral_api_key}"
}

"""
# mode: [{"role": "user", "content": "say yes"}]

    # Define the request body
    request_body = {
      "model": "mistral-tiny",
      "messages": [{"role": "user", "content": user_input}]
    }

    # Send the request
    response = requests.post(endpoint_url, headers=headers, json=request_body)
"""

# conversation-history setup
context_history = []


def add_to_context_history(role, comment):

    if role == 'user':
        segment = {"role": "user", "content": comment}

    elif role == 'assistant':
        segment = {"role": "assistant", "content": comment}

    else:
        print("add_to_context_history(role, comment)")
        print(role, comment)
        print('error')

    return segment


def prompt_user(user_input, context_history):

    context_history.append( add_to_context_history("user", user_input) )

    return context_history


def ask_mistral_tiny(context_history):
    # Define the request body
    request_body = {
      "model": "mistral-tiny",
      "messages": context_history
    }

    # Send the request
    response = requests.post(endpoint_url, headers=headers, json=request_body)

    # Check the response status code
    if response.status_code != 200:
      raise Exception(f"Error: {response.status_code} {response.text}")

    return response

def print_rec_ai(response, context_history):

    # Get the response data
    response_data = response.json()

    # Print the Mistral response

    ##
    ##
    # Turn this print on to see full return data
    ##
    ##
    """
    e.g.
    {
      "id": "635cb8d445ujhe5546bb64e5e7",
      "object": "chat.completion",
      "created": 170hrjfjf7084,
      "model": "mistral-tiny",
      "choices": [
        {
          "index": 0,
          "message": {
            "role": "assistant",
            "content": "Enjoy your cup of tea!"
          },
          "finish_reason": "stop",
          "logprobs": null
        }
      ],
      "usage": {
        "prompt_tokens": 575,
        "total_tokens": 629,
        "completion_tokens": 54
      }
    }
    """
    # print(json.dumps(response_data, indent=2))
    # print(type(response_data))

    output = response_data
    # print(type(output))
    # print(type(output["choices"][0]))

    # extract just the 'what they said' part out
    assistant_says = output["choices"][0]['message']['content']

    # print(assistant_says)

    new_comment = {"role": "assistant", "content": assistant_says}

    # add what assistant said to context history
    context_history.append(new_comment)

    return assistant_says, context_history


def go_user(user_input, context_history):
    """
    Input: context_history
    Ouput Tuple: assistant_says, context_history
    """

    # prompt user
    context_history = prompt_user(user_input, context_history)

    # prompt assistant
    response = ask_mistral_tiny(context_history)

    # ETL: Extract, Transform, & Load
    assistant_says, context_history = print_rec_ai(response, context_history)

    return assistant_says, context_history


def strip_non_alpha(text):
    # regex to leave only a-z characters
    pattern = re.compile('[^a-z]')
    return pattern.sub('', text).lower()


def keep_talking(context_history):
    """
    A very minimal chat with memory.

    Uses:
      query(input_string)
      strip_non_alpha(text)
    """
    still_talking = True
    dialogue_history = ""

    while still_talking:

        user_input = input("Say...")

        exit_phrase_list = [
            "exit",
            "quit",
            "quite",
            "!q",
            "q",
            "done",
            "finish",
            "end",
            "bye",
            "good bye",
        ]

        # check if user is exiting convesation
        if strip_non_alpha(user_input) in exit_phrase_list:
            print("\nAll Done!")
            break

        else:
            assistant_says, context_history = go_user(user_input, context_history)

            print( assistant_says )

            # save dialogue so far
            dialogue_history = context_history

    # when out of loop, return history
    return dialogue_history


# save history
def record_history_save_files(dialogue_history):

    date_time = dt.utcnow()
    timestamp = date_time.strftime('%Y/%m/%d  %H:%M:%S:%f')
    clean_timestamp = date_time.strftime('%Y%m%d%H%M')

    # To save the data directly as a JSON file:

    # Convert the Python dictionary list to a JSON string
    json_data = json.dumps(dialogue_history)

    # Open a file for writing in JSON format
    with open(f'json_dialog_{clean_timestamp}.json', 'w') as json_file:
        # Write the JSON string to the file
        json_file.write(json_data)


    # To save the data as a file readable as a script:

    # Create a new file for writing
    with open(f'script_dialog_{clean_timestamp}.txt', 'w') as script_file:

        # add timestamp
        text = timestamp + "\n\n"
        script_file.write(text)

        # Iterate over the dictionary list
        for item in dialogue_history:
            # Write the role and content of each item to the file, separated by a newline
            script_file.write(f"{item['role']}: {item['content']}\n\n")

In [52]:
# Reset History
context_history = []

In [ ]:
from datetime import datetime as dt

# make readable time
date_time = dt.utcnow()
timestamp = date_time.strftime('%Y/%m/%d  %H:%M:%S:%f')
clean_timestamp = date_time.strftime('%Y%m%d%H%M%S%f')

instructions = f"""
  Instructions: {timestamp}
    - Enter your queries into the iput-box.
    - Say 'quit' or 'exit', etc.,  to leave the conversation.

"""
print( instructions )

# run chat
dialogue_history = keep_talking(context_history)

# save history files
record_history_save_files(dialogue_history)